<a href="https://colab.research.google.com/github/waveology/aire/blob/main/5_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión de datos

Vamos a explorar algunas posibilidades de análisis de datos de calidad del aire que ofrece Python. Usaremos como hasta ahora datos meteorológicos de AEMET y de contaminación de la Comunidad de Madrid.

###1. Copia del repositorio de datos
---

Descargamos el repositorio de código y datos para trabajar más cómodamente:

In [ ]:
# Directorio de trabajo en Colab 
# ------------------------------------------------------------
%cd /content

# Si existe una copia previa del repositorio, la borramos:
# ----------------------------------------------------------------------------
!  rm -rf aire

# Creamos una copia del repositorio SOLO si no existe previamente
# ----------------------------------------------------------------------------
! [ ! -d aire ] && git clone https://github.com/waveology/aire.git

# Entramos en el repositorio que acabamos de copiar
# --------------------------------------------------
%cd aire

Importamos las extensiones que vamos a necesitar. 

Para simplificar la tarea hemos empaquetado las funciones de lectura de datos en un fichero independiente (lectura_de_datos.py) 

In [ ]:
import lectura_de_datos                                   # lee ficheros de datos meteorológicos y de contaminación de Madrid
import matplotlib.pyplot as plt                           # dibujo de gráficos
from matplotlib.dates import MonthLocator, DateFormatter  # formato de fechas 
from scipy import stats                                   # cálculo estadístico
import numpy as np                                        # matrices
import pandas as pd                                       # dataframes
from sklearn.linear_model import LinearRegression         # regresión
from sklearn.preprocessing import PolynomialFeatures      # regresión

###2. Inventario de magnitudes
---

####Estaciones de medida
**Código | Municipio | Nombre**

---
*       28005002   :    (  5,  'ALCALÁ DE HENARES'), 
*       28006004   :    (  6,  'ALCOBENDAS'), 
*       28007004   :    (  7,  'ALCORCÓN'), 
*       28009001   :	(  9,  'ALGETE'), 
*       28013002   :	( 13,  'ARANJUEZ'), 
*       28014002   :	( 14,  'ARGANDA DEL REY'),
*       28016001   :	( 16,  'EL ATAZAR'),
*       28045002   :	( 45,  'COLMENAR VIEJO'), 
*       28047002   :	( 47,  'COLLADO VILLALBA'), 
*       28049003   :	( 49,  'COSLADA'), 
*       28058004   :	( 58,  'FUENLABRADA'), 
*       28065014   :	( 65,  'GETAFE'), 
*       28067001   :	( 67,  'GUADALIX DE LA SIERRA'), 
*       28074007   :	( 74,  'LEGANÉS'), 
*       28080003   :	( 80,  'MAJADAHONDA'), 
*       28092005   :	( 92,  'MÓSTOLES'), 
*       28102001   :	(102,  'ORUSCO DE TAJUÑA'), 
*       28120001   : 	(120,  'PUERTO DE COTOS'), 
*       28123002   :	(123,  'RIVAS-VACIAMADRID'), 
*       28133002   :	(133,  'SAN MARTÍN DE VALDEIGLESIAS'), 
*       28148004   :	(148,  'TORREJÓN DE ARDOZ'), 
*       28161001   :	(161,  'VALDEMORO'), 
*       28171001   :	(171,  'VILLA DEL PRADO'), 
*       28180001   :	(180,  'VILLAREJO DE SALVANÉS')


####Contaminantes
**Código  |  Magnitud  | Unidades**

  
---
*      1 	:  ('Dióxido de azufre', 'μg/m³'),
*      6 	:  ('Monóxido de carbono', 'mg/m³'),
*      7 	:  ('Monóxido de nitrógeno', 'μg/m³'),
*      8 	:  ('Dióxido de nitrógeno', 'μg/m³'),
*      9 	:  ('Partículas en suspensión < PM2.5', 'μg/m³'),
*     10 	:  ('Partículas en suspensión < PM10',  'μg/m³'),
*     12 	:  ('Óxidos de nitrógeno', 'μg/m³'),
*     14 	:  ('Ozono', 'μg/m³'),
*     20 	:  ('Tolueno', 'μg/m³'),
*     22 	:  ('Black Carbon', 'μg/m³'),
*     30 	:  ('Benceno', 'μg/m³'),
*     42 	:  ('Hidrocarburos totales', 'mg/m³'),
*     44 	:  ('Hidrocarburos no metánicos', 'mg/m³'),
*    431  :  ('MetaParaXileno', 'μg/m³')
    }

####Meteorología
**Código | Magnitud | Unidades**

---
*   81 :	('Velocidad del viento',     'm/s'), 
*   82 :	('Dirección del viento',     'º'), 
*   83 :	('Temperatura',              'ºC'), 
*   86 :	('Humedad relativa',         '%'), 
*   87 :	('Presión atmosférica',      'hPa'), 
*   88 :	('Radiación solar',          'W/m2'), 
*   89 :	('Precipitación',            'mm')

###3. Carga de datos
---

Vamos a cargar datos de dos magnitudes para estudiar posibles relaciones:


In [ ]:

# # Ejemplo con datos meteorológicos
# # --------------------------------
# anio = 2022
# col1 = 'temperatura'
# col2 = 'humedad'
# df1, magnitud1, unidades1,estacion1 = lectura_de_datos.meteo(
#                                     'datos/meteo/%s.csv' % anio,
#                                      codigo_magnitud = 83,        # Temperatura
#                                      codigo_estacion = 28092005   # Guadalix de la Sierra  
#                                      ) 
# df2, magnitud2, unidades2,estacion2 = lectura_de_datos.meteo(
#                                     'datos/meteo/%s.csv' % anio,
#                                      codigo_magnitud = 86 ,        # Humedad relativa
#                                      codigo_estacion = 28092005    # Guadalix de la Sierra  
#                                      ) 


# Ejemplo con datos de contaminación
# ----------------------------------
anio = 2021
col1 = 'pm2.5'
col2 = 'pm10'
df1, magnitud1, unidades1,estacion1 = lectura_de_datos.comunidad(
                                    'datos/comunidad/%s.csv' % anio,
                                     codigo_magnitud = 9,          # PM2.5
                                     codigo_estacion = 28065014    # Getafe
                                     ) 
df2, magnitud2, unidades2,estacion2 = lectura_de_datos.comunidad(
                                    'datos/comunidad/%s.csv' % anio,
                                     codigo_magnitud = 10 ,        # PM10
                                     codigo_estacion = 28065014    # Getafe
                                     ) 

# Asignamos un nombre significativo a la columna 'valor'
# -------------------------------------------------------
df1.rename(columns={'valor':col1}, inplace=True)
df2.rename(columns={'valor':col2}, inplace=True)

# Fusionamos ambas series de datos en un mismo dataframe
# --------------------------------------------------------
df = pd.merge(df1, df2, left_index=True, right_index=True)

print(df.describe())

###4. Visualización de series temporales
---

Echamos un vistazo a ambas series temporales:

In [ ]:
# Ambas series en el mismo gráfico
# ---------------------------------
df.plot(
        marker='o',                                           # Símbolo
        ms=1,                                                 # Tamaño del símbolo
        lw=0,                                                 # Grosor de líneas de conexión
        grid=True,                                            # Rejilla
        figsize=(12,8),                                       # Tamaño del gráfico
        legend=True,                                          # Leyenda
        title='%s VS %s   -    %s  -  %s' % (magnitud2,magnitud1,estacion1,anio),                 # Titulo        
        xlabel= '%s %s' % (magnitud1,unidades1),              # Etiqueta X   
        ylabel= '%s %s' % (magnitud2,unidades2)               # Etiqueta Y   
        )
plt.show()

# # Cada serie en un gráfico independiente
# # ---------------------------------------
# fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(12,8))
# ax[0].plot(df[col1])
# ax[1].plot(df[col2])
# for a in ax :
#    a.grid(True)
#    a.set_ylim(0,200)
# plt.show()

###5. Comparación de series
---

Representamos una serie de datos respecto a la otra:

In [ ]:
ax = df.plot(x=col1, y=col2,
    marker='o',                                           # Símbolo
    ms=3,                                                 # Tamaño del símbolo
    lw=0,                                                 # Grosor de líneas de conexión
    color='blue',                                         # Color
    grid=True,                                            # Rejilla
    figsize=(12,8),                                       # Tamaño del gráfico
    legend=False,                                         # Leyenda
    title='%s vs %s   -    %s  -  %s' % (magnitud2,magnitud1,estacion1,anio),                 # Titulo        
    xlabel= '%s %s' % (magnitud1,unidades1),              # Etiqueta X   
    ylabel= '%s %s' % (magnitud2,unidades2),              # Etiqueta Y   
    xlim=(0,200),
    ylim=(0,200)
)
ax.set_aspect('equal')
plt.show()

###6. Regresión a una recta
---

Creamos un modelo lineal para ajustar los datos

In [ ]:
x = np.array(df[col1]).reshape(-1,1)  # Convierte una fila en una columna
y = np.array(df[col2])

In [ ]:
 model     = LinearRegression().fit(x,y)
 ajuste    = model.predict(x)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x,y,
        marker = 'o',
        ms     =  3,
        lw     =  0,
        color  = 'blue'
        )
ax.plot(x,ajuste, ls='-', lw=3, color='red')
ax.grid(True)
ax.set_title('%s VS %s   -    %s  -  %s' % (magnitud2,magnitud1,estacion1,anio)) # Titulo        
ax.set_xlabel('%s %s' % (magnitud1,unidades1))                                   # Etiqueta X   
ax.set_ylabel('%s %s' % (magnitud2,unidades2))                                   # Etiqueta Y 
plt.show()  

El coeficiente de determinación $R^{2}$ toma valores entre cero y uno y mide la calidad del ajuste:

In [ ]:
r2 = model.score(x,y)
print('R\u00b2 = %f' % r2)

Podemos obtener el valor en el que la recta de ajuste intersecta el eje y:

In [ ]:
b = model.intercept_
print('b = %f' % b)

Y también la pendiente de la recta:

In [ ]:
a = model.coef_
print('a = %f' % a[0])

De manera que nuestra recta de ajuste viene dada por:

### <center>$Y  = a * X + b$</center>

Podemos incorporar esta información al gráfico:

In [ ]:
info_ajuste  = 'Y = %.3f * X + %.3f\nR$^{2}=%.3f$' % (a,b,r2)
titulo       = '\n%s VS %s   -    %s  -  %s\n%s' % (magnitud2,magnitud1,estacion1,anio,info_ajuste)

fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x,y,
        marker = 'o',
        ms     =  3,
        lw     =  0,
        color  = 'blue'
        )
ax.plot(x,ajuste, ls='-', lw=3, color='red')
ax.grid(True)
ax.set_title(titulo, loc='center')                             # Titulo        
ax.set_xlabel('%s %s' % (magnitud1,unidades1))   # Etiqueta X   
ax.set_ylabel('%s %s' % (magnitud2,unidades2))   # Etiqueta Y 
plt.show()  

###7. Regresión a un polinomio
---

Vamos a aumentar los grados de libertar del ajuste usando un polinomio de grado arbitrario:

### <center>$Y = a\ X^{3} + b\ X^{2} + c\ X + d$</center>

In [ ]:

# Volvemos a extraer las columnas de datos del dataframe
# -------------------------------------------------------
x = np.array(df[col1])
y = np.array(df[col2])

# Regresión polinomial de grado n
# -------------------------------
poly = PolynomialFeatures(degree=3, include_bias=False)

# Preparación de datos
# ---------------------
poly_features = poly.fit_transform(x.reshape(-1, 1))

# Ajuste
# -------
model = LinearRegression()
model.fit(poly_features,y)

ajuste = model.predict(poly_features)
print(model.intercept_, model.coef_)

In [ ]:
# Estos son los coeficientes del polinomio
c, b, a   = model.coef_
d         = model.intercept_

xn = np.arange(x.min(),x.max(),step=1)
polinomio = a * xn**3 + b * xn**2 + c * xn + d

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x,y,
        marker = 'o',
        ms     = 3,
        lw     = 0,
        color  = 'blue'
        )
ax.plot(xn,polinomio, marker=None, ms=0, lw=3,color='red')
ax.grid(True)
poli = 'Y = (%.4f) * x$^{3}$ + (%.4f) * x$^{2}$ + (%.4f) * X + (%.4f)' % (a,b,c,d)
ax.set_title('%s VS %s   -    %s  -  %s\n%s' % (magnitud2,magnitud1,estacion1,anio,poli)) # Titulo        
ax.set_xlabel('%s %s' % (magnitud1,unidades1))                                   # Etiqueta X   
ax.set_ylabel('%s %s' % (magnitud2,unidades2))                                   # Etiqueta Y  
plt.show()

###8. Eliminación de datos atípicos
---

¿Cómo cambian estos resultados si eliminamos los datos **atípicos**?

In [ ]:
# Calculamos los percentiles 25 y 75 de la primera columna
# ---------------------------------------------------------
p25_x, p75_x = df['pm2.5'].quantile((0.25,0.75))

# Calculamos el rango intercuartílico de la primera columna
# --------------------------------------------------------
riq_x = p75_x - p25_x

# Calculamos los percentiles 25 y 75 de la segunda  columna
# ---------------------------------------------------------
p25_y, p75_y = df['pm10'].quantile((0.25,0.75))

# Calculamos el rango intercuartílico de la primera columna
# --------------------------------------------------------
riq_y = p75_y - p25_y

# Eliminamos los valores que se encuentran más allá
# de 1.5 veces el rango intercuartílico
# ---------------------------------------------------
df = df[
        (df['pm2.5'] < 1.5 * (p75_x + riq_x)) & 
        (df['pm2.5'] > 1.5 * (p25_x - riq_x)) & 
        (df['pm10']  < 1.5 * (p75_y + riq_y)) & 
        (df['pm10']  > 1.5 * (p25_y - riq_y))
   ]